In [1]:
import googlemaps
import dotenv
import time
import os
import pandas as pd

# debug
from pprint import pprint

In [2]:
%load_ext dotenv
chave_api = os.getenv('API_KEY')
client = googlemaps.Client(chave_api)

In [20]:
name_sheet = 'Lojas'
planilha_qq = pd.read_excel(
    'Quero-Quero-RI.xlsx',
    sheet_name= name_sheet,
    skiprows = 2,
    usecols = ['Cidade (UF)']
)

cidades = planilha_qq['Cidade (UF)'].unique()

lista_coordenadas = []
for i, cidade in enumerate(cidades):
    response = client.geocode(cidade)

    if len(response) < 1:
        time.sleep(.5)
        pass
    else:

        latitude = response[0]['geometry']['location']['lat']
        longitude = response[0]['geometry']['location']['lng']

        coordenadas = (latitude, longitude)
        lista_coordenadas.append(coordenadas)
        time.sleep(.15)
    print(len(cidades) - i, coordenadas)

449 (-27.8264491, -54.6630924)
448 (-27.9480354, -54.7530498)
447 (-28.1437464, -54.7192376)
446 (-27.9067345, -55.1386175)
445 (-27.6585821, -54.44177579999999)
444 (-27.7603566, -54.4790529)
443 (-28.0222723, -54.9382192)
442 (-27.5016088, -54.09907)
441 (-28.1459864, -54.5565401)
440 (-27.628239, -54.30597280000001)
439 (-27.4559325, -53.9301575)
438 (-27.6687078, -54.1115258)
437 (-27.7314024, -52.9957759)
436 (-27.3718047, -53.7584343)
435 (-27.8700347, -54.4797042)
434 (-28.4081313, -54.9615193)
433 (-27.7841041, -54.2344607)
432 (-28.3001866, -54.26583170000001)
431 (-28.3891409, -53.9220126)
430 (-28.659954, -56.0065719)
429 (-27.9013019, -53.311712)
428 (-27.8508886, -53.7763232)
427 (-28.460032, -52.82085559999999)
426 (-28.0283749, -54.3508179)
425 (-28.6452209, -53.60523449999999)
424 (-27.940972, -52.921729)
423 (-27.991133, -54.8405182)
422 (-29.1282237, -56.5552712)
421 (-28.6229959, -52.8711249)
420 (-27.3587196, -53.3993835)
419 (-27.5099828, -54.3630548)
418 (-27.2313

In [17]:
len(response)

0

In [22]:
lista_dfs = []

for i, coordenada in enumerate(lista_coordenadas):
    print(coordenada, len(lista_coordenadas) - i)
    response = client.places_nearby(location = coordenada, keyword = 'Quero-Quero', radius = 50000)

    while True:
        for store in response['results']:
            df = pd.DataFrame({'nome': store['name'],
            'endereco': store['vicinity'],
            'latitude': store['geometry']['location']['lat'],
            'longitude': store['geometry']['location']['lng']}, index = [0])
            
            lista_dfs.append(df)
        
        if 'next_page_token' in response:
            next_page_token = response['next_page_token']
            time.sleep(3)
            response = client.places_nearby(
                location= coordenada,
                keyword= 'Quero-Quero',
                radius = 50000,
                page_token = next_page_token
            )
        else:
            break

    time.sleep(.2)

lojas_qq = pd.concat(lista_dfs, ignore_index=True)

(-27.8264491, -54.6630924) 447
(-27.9480354, -54.7530498) 446
(-28.1437464, -54.7192376) 445
(-27.9067345, -55.1386175) 444
(-27.6585821, -54.44177579999999) 443
(-27.7603566, -54.4790529) 442
(-28.0222723, -54.9382192) 441
(-27.5016088, -54.09907) 440
(-28.1459864, -54.5565401) 439
(-27.628239, -54.30597280000001) 438
(-27.4559325, -53.9301575) 437
(-27.6687078, -54.1115258) 436
(-27.7314024, -52.9957759) 435
(-27.3718047, -53.7584343) 434
(-27.8700347, -54.4797042) 433
(-28.4081313, -54.9615193) 432
(-27.7841041, -54.2344607) 431
(-28.3001866, -54.26583170000001) 430
(-28.3891409, -53.9220126) 429
(-28.659954, -56.0065719) 428
(-27.9013019, -53.311712) 427
(-27.8508886, -53.7763232) 426
(-28.460032, -52.82085559999999) 425
(-28.0283749, -54.3508179) 424
(-28.6452209, -53.60523449999999) 423
(-27.940972, -52.921729) 422
(-27.991133, -54.8405182) 421
(-29.1282237, -56.5552712) 420
(-28.6229959, -52.8711249) 419
(-27.3587196, -53.3993835) 418
(-27.5099828, -54.3630548) 417
(-27.2313442,

ValueError: No objects to concatenate

In [24]:
type(lista_dfs)

list